# Importing Libraries

In [2]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Setting up device diagnostic code, to run our code on gpu directly

In [3]:
device = "cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

In [4]:
df = pd.read_csv("D:/Codes/Pycharm Projects/PYTORCH/PROJECTS/AI Healthcare/AI-IN-HEALTHCARE/Case_Study - 1 [ Predicting Disease Outcomes from Patient Data ]/Data/hospital data analysis.csv")
df

,Patient_ID,Age,Gender,Condition,Procedure,Cost,Length_of_Stay,Readmission,Outcome,Satisfaction
0,1,45,Female,Heart Disease,Angioplasty,15000,5,No,Recovered,4
1,2,60,Male,Diabetes,Insulin Therapy,2000,3,Yes,Stable,3
2,3,32,Female,Fractured Arm,X-Ray and Splint,500,1,No,Recovered,5
3,4,75,Male,Stroke,CT Scan and Medication,10000,7,Yes,Stable,2
4,5,50,Female,Cancer,Surgery and Chemotherapy,25000,10,No,Recovered,4
...,...,...,...,...,...,...,...,...,...,...
979,996,68,Male,Hypertension,Medication and Counseling,1000,70,No,Stable,4
980,997,45,Female,Appendicitis,Appendectomy,8000,72,No,Recovered,3
981,998,40,Male,Fractured Leg,Cast and Physical Therapy,3000,72,No,Recovered,4
982,999,78,Female,Heart Attack,Cardiac Catheterization,18000,74,Yes,Stable,2


In [5]:
df.dtypes

Patient_ID         int64
Age                int64
Gender            object
Condition         object
Procedure         object
Cost               int64
Length_of_Stay     int64
Readmission       object
Outcome           object
Satisfaction       int64
dtype: object

In [6]:
unique_condition = df['Condition'].unique()
for condition in unique_condition:
    print(condition)

Heart Disease
Diabetes
Fractured Arm
Stroke
Cancer
Hypertension
Appendicitis
Fractured Leg
Heart Attack
Allergic Reaction
Respiratory Infection
Prostate Cancer
Childbirth
Kidney Stones
Osteoarthritis


In [7]:
outcome_by_condition = df.groupby(['Condition','Outcome']).size().unstack(fill_value = 0)
outcome_by_condition

Outcome,Recovered,Stable
Condition,,
Allergic Reaction,66,0
Appendicitis,66,0
Cancer,66,0
Childbirth,65,0
Diabetes,0,65
Fractured Arm,66,0
Fractured Leg,67,0
Heart Attack,0,67
Heart Disease,65,0


In [8]:
outcome_by_condition = df.groupby(['Age','Condition','Outcome']).size().unstack(fill_value = 0)
outcome_by_condition

Outcome                    Recovered  Stable
Age Condition                               
25  Allergic Reaction             34       0
28  Allergic Reaction             32       0
30  Childbirth                     1       0
    Fractured Arm                 32       0
32  Childbirth                    32       0
    Fractured Arm                  1       0
35  Childbirth                    32       0
    Fractured Arm                 33       0
40  Fractured Leg                 34       0
45  Appendicitis                  32       0
    Fractured Leg                 33       0
    Heart Disease                  1       0
48  Heart Disease                 31       0
    Respiratory Infection          0       1
50  Cancer                         1       0
    Kidney Stones                 32       0
52  Kidney Stones                  1       0
    Respiratory Infection          0      64
53  Cancer                        33       0
55  Appendicitis                   1       0
    Cancer                        32       0
    Heart Disease                 33       0
    Kidney Stones                 32       0
58  Osteoarthritis                 0       1
    Prostate Cancer               32       0
60  Appendicitis                  33       0
    Diabetes                       0       1
    Osteoarthritis                 0      31
62  Diabetes                       0      33
65  Diabetes                       0      31
    Osteoarthritis                 0      32
    Prostate Cancer                1       0
67  Prostate Cancer               32       0
68  Hypertension                   0      33
70  Heart Attack                   0       1
    Stroke                         0      32
72  Hypertension                   0      33
75  Heart Attack                   0      33
    Stroke                         0       1
78  Heart Attack                   0      33
    Stroke                         0      33

In [9]:
df1 = df
df1.drop('Patient_ID', axis = 'columns', inplace = True)
df1.shape

(984, 9)

In [12]:
df1.dtypes

Age                int64
Gender            object
Condition         object
Procedure         object
Cost               int64
Length_of_Stay     int64
Readmission        int64
Outcome           object
Satisfaction       int64
dtype: object

In [10]:
def condition_per_outcome_object(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f"{column} : {df[column].unique()}")

condition_per_outcome_object(df1)

Gender : ['Female' 'Male']
Condition : ['Heart Disease' 'Diabetes' 'Fractured Arm' 'Stroke' 'Cancer'
 'Hypertension' 'Appendicitis' 'Fractured Leg' 'Heart Attack'
 'Allergic Reaction' 'Respiratory Infection' 'Prostate Cancer'
 'Childbirth' 'Kidney Stones' 'Osteoarthritis']
Procedure : ['Angioplasty' 'Insulin Therapy' 'X-Ray and Splint'
 'CT Scan and Medication' 'Surgery and Chemotherapy'
 'Medication and Counseling' 'Appendectomy' 'Cast and Physical Therapy'
 'Cardiac Catheterization' 'Epinephrine Injection' 'Antibiotics and Rest'
 'Radiation Therapy' 'Delivery and Postnatal Care' 'Lithotripsy'
 'Physical Therapy and Pain Management']
Readmission : ['No' 'Yes']
Outcome : ['Recovered' 'Stable']


In [11]:
yes_no_columns = ['Readmission']
for column in yes_no_columns:
    df1[column].replace({"Yes" : 1, "No" : 1}, inplace = True)

def condition_per_outcome(df):
    for column in df:
        print(f"{column} : {df[column].unique()}")
condition_per_outcome(df1)

Age : [45 60 32 75 50 68 55 40 70 25 48 65 30 52 58 62 35 78 53 72 28 67]
Gender : ['Female' 'Male']
Condition : ['Heart Disease' 'Diabetes' 'Fractured Arm' 'Stroke' 'Cancer'
 'Hypertension' 'Appendicitis' 'Fractured Leg' 'Heart Attack'
 'Allergic Reaction' 'Respiratory Infection' 'Prostate Cancer'
 'Childbirth' 'Kidney Stones' 'Osteoarthritis']
Procedure : ['Angioplasty' 'Insulin Therapy' 'X-Ray and Splint'
 'CT Scan and Medication' 'Surgery and Chemotherapy'
 'Medication and Counseling' 'Appendectomy' 'Cast and Physical Therapy'
 'Cardiac Catheterization' 'Epinephrine Injection' 'Antibiotics and Rest'
 'Radiation Therapy' 'Delivery and Postnatal Care' 'Lithotripsy'
 'Physical Therapy and Pain Management']
Cost : [15000  2000   500 10000 25000  1000  8000  3000 18000   100   800 20000
 12000  6000  4000]
Length_of_Stay : [ 5  3  1  7 10  2  4  6  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 5

C:\Users\Sanket S Dharne\AppData\Local\Temp\ipykernel_25772\2662095484.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1[column].replace({"Yes" : 1, "No" : 1}, inplace = True)
C:\Users\Sanket S Dharne\AppData\Local\Temp\ipykernel_25772\2662095484.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True